# Atividade 1 - Análise Preditiva com o Spark

Discente: Tiago César da Silva Lopes

Data: 16/11/23

## Informações sobre o projeto

data set: calculo_renal.csv

Este conjunto de dados pode ser usado para prever a presença de cálculos renais com base na análise de urina. As amostras de urina foram analisadas para
determinar se certas características físicas da urina podem estar relacionadas com a formação de cristais de oxalato de cálcio.

Dicionário:
As seis características físicas da urina são:
id - ID do registro
gravidade -  gravidade específica;
ph - pH;  
osmo - osmolaridade;
cond - condutividade (mMho miliMho). A condutividade é proporcional à concentração de carga íons em solução;
ureia - concentração de ureia em milimoles por litro;
calcio - cálcio concentração (CALC) em milimoleslitro;
saida - existência de calculo renal



## Instalação e exibição inicial do dataset

In [1]:
!pip install pyspark

In [2]:
# Importar as bibliotecas necessárias
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

Iniciando uma sessão local e importando os dados:


In [3]:
# Iniciar uma sessão local e importar dados
from pyspark.sql import SparkSession
sc = SparkSession.builder.master('local[*]').getOrCreate()

In [4]:
# carregar dados do meu computador
dados_spark = sc.read.csv("calculo_renal.csv", header=True)

# Verificando o tipo de objeto criado
type(dados_spark)

pyspark.sql.dataframe.DataFrame

In [5]:
# Verificando o dataset
dados_spark.show(5)

+---+---------+----+----+----+-----+------+-----+
| id|gravidade|  ph|osmo|cond|ureia|calcio|saida|
+---+---------+----+----+----+-----+------+-----+
|  0|    1.013|6.19| 443|14.8|  124|  1.45|    0|
|  1|    1.025| 5.4| 703|23.6|  394|  4.18|    0|
|  2|    1.009|6.13| 371|24.5|  159|  9.04|    0|
|  3|    1.021|4.91| 442|20.8|  398|  6.63|    1|
|  4|    1.021|5.53| 874|17.8|  385|  2.21|    1|
+---+---------+----+----+----+-----+------+-----+
only showing top 5 rows



In [6]:
# Verificando o schema() deste spark dataframe
dados_spark.printSchema()

root
 |-- id: string (nullable = true)
 |-- gravidade: string (nullable = true)
 |-- ph: string (nullable = true)
 |-- osmo: string (nullable = true)
 |-- cond: string (nullable = true)
 |-- ureia: string (nullable = true)
 |-- calcio: string (nullable = true)
 |-- saida: string (nullable = true)



In [7]:
#Retornar o número de linhas
dados_spark.count()

414

## Análise inicial da base de dados

Podemos verificar as médias de algumas colunas e a quantidades de saídas positivas e negativas que existe.

In [8]:
#Principais estatísticas
dados_spark.describe('ph').show(5)

+-------+-----------------+
|summary|               ph|
+-------+-----------------+
|  count|              414|
|   mean|5.955458937198068|
| stddev|0.642260050825236|
|    min|             4.76|
|    max|             7.94|
+-------+-----------------+



In [9]:
dados_spark.describe('ureia').show(5)

+-------+------------------+
|summary|             ureia|
+-------+------------------+
|  count|               414|
|   mean|278.65700483091786|
| stddev|136.44224930839462|
|    min|                10|
|    max|                95|
+-------+------------------+



In [10]:
dados_spark.describe('calcio').show(5)

+-------+------------------+
|summary|            calcio|
+-------+------------------+
|  count|               414|
|   mean| 4.114637681159421|
| stddev|3.2176405021529146|
|    min|              0.17|
|    max|              9.39|
+-------+------------------+



In [11]:
# Importar a função "col" para trabalhar com colunas
from pyspark.sql.functions import col

#importar sql: groupBy
from pyspark.sql import functions as F

# Contar as ocorrências únicas e ordenar os resultados
dados_spark.groupBy("saida").count().orderBy(col("saida").desc()).show(truncate=False)

+-----+-----+
|saida|count|
+-----+-----+
|1    |184  |
|0    |230  |
+-----+-----+



## Análise e extração de informações

### Quais foram as ocorrências de PH?

In [12]:
ocorrencias_ph = dados_spark.groupBy('ph').agg(F.count('ph').alias('count(ph)')).orderBy(F.desc('count(ph)')).show(truncate=False)

+----+---------+
|ph  |count(ph)|
+----+---------+
|5.53|34       |
|5.68|16       |
|6.13|13       |
|5.58|13       |
|6.19|12       |
|6.79|11       |
|5.87|11       |
|5.71|10       |
|5.94|9        |
|5.24|9        |
|5.51|9        |
|6.29|9        |
|6.76|9        |
|5.62|9        |
|6.28|8        |
|5.21|8        |
|7.38|8        |
|5.41|8        |
|5.66|8        |
|5.44|8        |
+----+---------+
only showing top 20 rows



### Como foi as ocorrências de Cálcio dos pacientes?

In [13]:
ocorrencias_calcio = dados_spark.groupBy('calcio').agg(F.count('calcio').alias('count(calcio)')).orderBy(F.desc('count(calcio)')).show(truncate=False)

+------+-------------+
|calcio|count(calcio)|
+------+-------------+
|4.49  |23           |
|1.16  |17           |
|1.27  |16           |
|2.45  |12           |
|3.98  |11           |
|3.34  |11           |
|12.68 |10           |
|8.48  |10           |
|2.21  |10           |
|2.31  |10           |
|8.94  |10           |
|6.96  |10           |
|3.46  |9            |
|6.99  |9            |
|9.39  |9            |
|2.64  |8            |
|1.53  |8            |
|4.18  |8            |
|3.01  |7            |
|0.83  |7            |
+------+-------------+
only showing top 20 rows



### Como foi as ocorrências de ureia?

In [14]:
ocorrencias_ureia = dados_spark.groupBy('ureia').agg(F.count('ureia').alias('count(ureia)')).orderBy(F.desc('count(ureia)')).show(truncate=False)

+-----+------------+
|ureia|count(ureia)|
+-----+------------+
|159  |16          |
|87   |14          |
|385  |13          |
|395  |13          |
|75   |11          |
|195  |10          |
|398  |10          |
|620  |10          |
|382  |10          |
|301  |10          |
|214  |9           |
|199  |9           |
|224  |9           |
|349  |9           |
|178  |9           |
|239  |9           |
|64   |8           |
|124  |8           |
|430  |8           |
|170  |8           |
+-----+------------+
only showing top 20 rows



Podemos perceber algumas ocorrências entre o PH, o cálcio e a ureia dos pacientes. Com o desenvolvimento das análises poderemos criar a relação entre os fatores e a saída do cálculo renal.

## Valores nulos

In [15]:
#Dados faltantes
dados_spark.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in dados_spark.columns]).show()

+---+---------+---+----+----+-----+------+-----+
| id|gravidade| ph|osmo|cond|ureia|calcio|saida|
+---+---------+---+----+----+-----+------+-----+
|  0|        0|  0|   0|   0|    0|     0|    0|
+---+---------+---+----+----+-----+------+-----+



Podemos perceber que a base de dados não possui nenhum valor nulo. Nesse caso, vamos prosseguir para o treinamento dos dados.

## Criando uma instância para trabalhar no Modelo de classificação

In [16]:
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType

# Inicializando a sessão Spark
spark = SparkSession.builder.appName("DecisionTree").getOrCreate()

#Definição do DataFrame com base nos tipos de dados do conjunto de dados
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("gravidade", DoubleType(), True),
    StructField("ph", DoubleType(), True),
    StructField("osmo", IntegerType(), True),
    StructField("cond", DoubleType(), True),
    StructField("ureia", IntegerType(), True),
    StructField("calcio", DoubleType(), True),
    StructField("saida", IntegerType(), True)
])

seu_caminho = '/content/calculo_renal.csv'

df = spark.read.csv(seu_caminho, header=True, schema=schema)

# Visualize os primeiros registros do DataFrame
df.show(5)


+---+---------+----+----+----+-----+------+-----+
| id|gravidade|  ph|osmo|cond|ureia|calcio|saida|
+---+---------+----+----+----+-----+------+-----+
|  0|    1.013|6.19| 443|14.8|  124|  1.45|    0|
|  1|    1.025| 5.4| 703|23.6|  394|  4.18|    0|
|  2|    1.009|6.13| 371|24.5|  159|  9.04|    0|
|  3|    1.021|4.91| 442|20.8|  398|  6.63|    1|
|  4|    1.021|5.53| 874|17.8|  385|  2.21|    1|
+---+---------+----+----+----+-----+------+-----+
only showing top 5 rows



In [17]:
from pyspark.ml.feature import VectorAssembler

# Criando o VectorAssembler
feature_cols = ["gravidade", "ph", "osmo", "cond", "ureia", "calcio"]
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

# Aplicando o VectorAssembler no DataFrame
data_with_features = assembler.transform(df)

# Resultado
data_with_features.show(5)


+---+---------+----+----+----+-----+------+-----+--------------------+
| id|gravidade|  ph|osmo|cond|ureia|calcio|saida|            features|
+---+---------+----+----+----+-----+------+-----+--------------------+
|  0|    1.013|6.19| 443|14.8|  124|  1.45|    0|[1.013,6.19,443.0...|
|  1|    1.025| 5.4| 703|23.6|  394|  4.18|    0|[1.025,5.4,703.0,...|
|  2|    1.009|6.13| 371|24.5|  159|  9.04|    0|[1.009,6.13,371.0...|
|  3|    1.021|4.91| 442|20.8|  398|  6.63|    1|[1.021,4.91,442.0...|
|  4|    1.021|5.53| 874|17.8|  385|  2.21|    1|[1.021,5.53,874.0...|
+---+---------+----+----+----+-----+------+-----+--------------------+
only showing top 5 rows



In [18]:
# Divida os dados em conjuntos de treinamento e teste
train_data, test_data = data_with_features.randomSplit([0.8, 0.2], seed=123)

# Inicialize o modelo de Árvore de Decisão
dt = DecisionTreeClassifier(featuresCol="features", labelCol="saida")

In [19]:
# Treine o modelo de Árvore de Decisão
model = dt.fit(train_data)

# Faça previsões no conjunto de teste
predictions = model.transform(test_data)

# Exiba as previsões e os valores reais
predictions.select("id", "gravidade", "ph", "osmo", "cond", "ureia", "calcio", "saida", "prediction").show()


+---+---------+----+----+----+-----+------+-----+----------+
| id|gravidade|  ph|osmo|cond|ureia|calcio|saida|prediction|
+---+---------+----+----+----+-----+------+-----+----------+
|  2|    1.009|6.13| 371|24.5|  159|  9.04|    0|       1.0|
|  6|    1.008|5.09| 371|15.5|  159|  2.17|    1|       0.0|
| 12|    1.017|6.79| 541|20.0|  187|  6.96|    1|       1.0|
| 13|    1.021|6.56| 703|22.2|  418|  4.18|    1|       1.0|
| 18|    1.018|6.13| 416|12.8|  178|  3.01|    0|       0.0|
| 23|    1.007|5.87| 704|23.6|  260|  1.53|    0|       0.0|
| 29|    1.015|6.61| 416|11.2|  178|  3.98|    0|       1.0|
| 30|     1.02|5.58| 668|25.3|  252|  2.33|    0|       0.0|
| 33|    1.021|5.53| 781|23.6|  349|  2.21|    0|       0.0|
| 34|    1.011|7.01| 443|25.7|  124|  1.45|    1|       0.0|
| 35|    1.009|6.29| 242|12.2|   87|  1.19|    0|       0.0|
| 38|    1.025|6.75| 840|32.6|  395|  4.54|    0|       1.0|
| 40|    1.015|5.41| 450|11.2|  170|  2.64|    0|       0.0|
| 47|    1.021|5.66|1032

In [20]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Avalie o modelo com precisão, recall e acurácia
evaluator = MulticlassClassificationEvaluator(labelCol="saida", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

precision_evaluator = MulticlassClassificationEvaluator(labelCol="saida", predictionCol="prediction", metricName="weightedPrecision")
precision = precision_evaluator.evaluate(predictions)

recall_evaluator = MulticlassClassificationEvaluator(labelCol="saida", predictionCol="prediction", metricName="weightedRecall")
recall = recall_evaluator.evaluate(predictions)

print("Acurácia do Modelo:", accuracy)
print("Precisão do Modelo:", precision)
print("Recall do Modelo:", recall)

# Encerre a sessão Spark
spark.stop()


Acurácia do Modelo: 0.6746987951807228
Precisão do Modelo: 0.6748947007542365
Recall do Modelo: 0.6746987951807228
